In [1]:
using Base.Test

In [2]:
using Match

In [3]:
_next_var_id=-1
immutable Var
    id::Int
end
function Var()
    global _next_var_id+=1
    Var(_next_var_id)
end

Var

 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in require at ./loading.jl:243
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl:177
 in eventloop at /home/ubuntu/.julia/v0.5/IJulia/src/IJulia.jl:141
 in anonymous at task.jl:443
while loading /home/ubuntu/.julia/v0.5/Match/src/matchmacro.jl, in expression starting on line 225
 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in require at ./loading.jl:243
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl:177
 in eventloop at /home/ubuntu/.julia/v0.5/IJulia/src/IJulia.jl:141
 in 

In [4]:
function walk(u::Any, s)
    u
end

function walk(u::Var,s)
    for (v,w) in s
        if v==u
            return walk(w,s)
        end
    end
    u #Otherwise
end

walk (generic function with 2 methods)

In [5]:
function ext_s(x,v,s)
    ((x,v),s)
end

ext_s (generic function with 1 method)

In [6]:
function unify(u,v,s)
    u = walk(u,s)
    v = walk(v,s)
    @match (u,v) begin
        (u::Var, v::Var), if u==v end => s
        (u::Var, v) => ext_s(u,v,s)
        (u,v::Var)  => ext_s(v,u,s)
        ((u1,u2),(v1,v2)) => begin 
            s=unify(u1,v1,s)
            s!=false ? unify(u2,v2,s) : false
        end
        (u,v), if(u==v) end => s
        _ => false
    end
end            

unify (generic function with 1 method)

In [14]:
unify((1,u),(1,2),[]) == ((u,2),[])

LoadError: LoadError: UndefVarError: u not defined
while loading In[14], in expression starting on line 1

Test Passed
  Expression: unify((1,u),(2,2),[]) == false
   Evaluated: false == false

In [9]:
function ≡(u,v)
    function λ(sc)
        s = unify(u,v, sc[1])
        if s!=False
            [s]
        else
            []
        end
    end
end       

≡ (generic function with 1 method)

In [17]:
using Base.Test
begin #Test Vars
    a=Var()
    b=Var()
    @test(a!=b)
    @test(a==a)
    @test typeof(a)==Var
end

begin #Test unification
    u=Var()
    @test unify((1,u),(1,2),[]) == ((u,2),[])
    @test unify((1,u),(2,2),[]) == false
end

Test Passed
  Expression: unify((1,u),(2,2),[]) == false
   Evaluated: false == false

In [25]:
import Base: start, next, done, eltype, length

immutable Interleave
    xss::Vector{Any}
    function Interleave(xss...)
        new(Any[xss...])
    end
end


"""
Interleave is the transpose of chain.
It returns one element from each of the iterators in turn.
If one of the interators runs dry then it is skippted
eg:
```
collect(interleave([1,2,3],[10,20,30,40,50],[0.1,0.2]))
>[1,10,0.1,2,20,0.2,3,30,40,50]
```
"""
function interleave(xss...)
    Interleave(xss...)
end

function start(it::Interleave)
    xs_states = map(start, it.xss)
    return 1, xs_states
end

function next(it::Interleave, state)
    i, xs_states = state
    
    #Find a nonempty list
    initial_i = i
    while done(it.xss[i], xs_states[i]) #Look for one that isn't done
        i += 1
        if i == length(it.xss)+ 1
            i=1
        end
        
        if i==initial_i
            #Then we have dones a full loop, and none are good
            throw(BoundsError("Next was called on Interleave, but all contained iterators are Done"))
        end
    end
    
    #find value to return
    v, xs_state = next(it.xss[i], xs_states[i])
    
    #update xs_state
    new_xs_states = copy(xs_states)
    new_xs_states[i]=xs_state 
    
    #Find next state
    new_i=i+1
    if new_i == length(it.xss)+ 1
        new_i=1
    end
    
    return v, (new_i, new_xs_states)
end

function done(it::Interleave, state)
    i, xs_states = state
    all(Bool[done(xs,xs_state) for (xs, xs_state) in zip(it.xss, xs_states)]) #can't use map as typeinference fails on map(done,[],[])
end

function eltype(it::Interleave)
    #Literally Identical to eltype(it::Chain), could be combined by using a typeunion there
    try
        typejoin([eltype(xs) for xs in it.xss]...)
    catch
        Any
    end
end

using Base.Test
@test collect(interleave()) == [] 
@test collect(interleave([1,2,3])) == [1,2,3]
@test collect(interleave([1,2,3],[10,20,30])) == [1,10,2,20,3,30]
@test collect(interleave([1,2,3],[10,20,30,40,50],[0.1,0.2])) == [1,10,0.1,2,20,0.2,3,30,40,50]

Test Passed
  Expression: collect(interleave([1,2,3],[10,20,30,40,50],[0.1,0.2])) == [1,10,0.1,2,20,0.2,3,30,40,50]
   Evaluated: Real[1,10,0.1,2,20,0.2,3,30,40,50] == [1.0,10.0,0.1,2.0,20.0,0.2,3.0,30.0,40.0,50.0]

In [ ]:
for i in interleave(1:3, ['a', 'b', 'c','d'])
    @show i
end

In [ ]:
done([],2)

In [ ]:
for (ii, v) in enumerate(interleave([1,2,3],[10,20,30,40,50],[0.1,0.2])) 
    println(ii," - ", v)
    if ii>20
        break
    end
end

In [ ]:
rem(5,6)